<a href="https://colab.research.google.com/github/itsokaypiyush/DOODLEDOJO/blob/main/TextToSpeech_Initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install dependencies
!pip install pymupdf transformers pydub edge-tts
!apt-get install ffmpeg -y

# STEP 2: Upload PDF or enter text
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import fitz

option = widgets.ToggleButtons(
    options=['Upload PDF', 'Type Text'],
    description='Choose Input:',
)
display(option)

text_box = widgets.Textarea(
    placeholder='Paste or type your content here...',
    layout=widgets.Layout(width='100%', height='200px')
)
display(text_box)

uploaded = None
raw_text = ""

import time
time.sleep(10)  # Give time to upload or type

if option.value == 'Upload PDF':
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    doc = fitz.open(filename)
    for page in doc:
        raw_text += page.get_text()
else:
    raw_text = text_box.value

print("✅ Text Ready")

# STEP 3: Summarize
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def chunk_text(text, max_len=900):
    import textwrap
    return textwrap.wrap(text, max_len)

chunks = chunk_text(raw_text)
summarized = []

for chunk in chunks:
    out = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
    summarized.append(out[0]['summary_text'])

final_summary = " ".join(summarized)
print("🧠 Summary:\n", final_summary)

# STEP 4: Ask Questions
qa_pipeline = pipeline("question-answering")

def ask(question):
    return qa_pipeline(question=question, context=raw_text)['answer']

print("❓ Example Q&A:")
print("Q: What is this text about?\nA:", ask("What is this text about?"))

# STEP 5: Podcast Script
def generate_script(summary):
    return f"""
🎙️ [Host - Jenny]: Welcome to our AI-driven summary!
🧠 [Guest - Guy]: Here's the core idea – {summary}
🎙️ [Host - Jenny]: That wraps it up. Stay curious!
"""

script = generate_script(final_summary)

# STEP 6: Voiceover
import edge_tts
import asyncio

lines = script.strip().split('\n')

voice_map = {
    'Jenny': 'en-US-JennyNeural',
    'Guy': 'en-US-GuyNeural'
}

async def make_voice(lines):
    voice_files = []
    for i, line in enumerate(lines):
        if ']: ' not in line: continue
        label, content = line.split(']: ', 1)
        speaker = label.split(' - ')[-1].strip('[] ')
        voice = voice_map.get(speaker, 'en-US-JennyNeural')
        file = f"line_{i}.mp3"
        tts = edge_tts.Communicate(content, voice)
        await tts.save(file)
        voice_files.append(file)
    return voice_files

voice_files = await make_voice(lines)

from pydub import AudioSegment

final_audio = AudioSegment.empty()
for file in voice_files:
    final_audio += AudioSegment.from_mp3(file)

final_audio.export("notebooklm_podcast.mp3", format="mp3")
files.download("notebooklm_podcast.mp3")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


ToggleButtons(description='Choose Input:', options=('Upload PDF', 'Type Text'), value='Upload PDF')

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Paste or type your content here..…

Saving Uni.pdf to Uni (4).pdf
✅ Text Ready


Device set to use cpu
Your max_length is set to 130, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


...

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


❓ Example Q&A:
Q: What is this text about?
A: Indian Institutes of 
Technology


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install necessary packages
!pip install pymupdf transformers requests pydub

from google.colab import files
import fitz
import requests
from transformers import pipeline
import IPython.display as ipd
from pydub import AudioSegment
import os

# STEP 1: Upload PDF
print("Upload your PDF file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# STEP 2: Extract text from PDF
doc = fitz.open(filename)
print("Extracting text from PDF...")
text = ""
for page in doc:
    text += page.get_text()
print(f"Extracted {len(text)} characters of text.")

# STEP 3: Initialize summarizer pipeline
print("Loading summarization model...")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Helper function: chunk text for summarization
def chunk_text(text, max_len=1000):
    paragraphs = text.split('\n')
    chunks = []
    current_chunk = ""
    for para in paragraphs:
        if len(current_chunk) + len(para) < max_len:
            current_chunk += para + "\n"
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = para + "\n"
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# STEP 4: Summarize the text in chunks
print("Splitting text into chunks for summarization...")
chunks = chunk_text(text, max_len=1000)
print(f"Total chunks to summarize: {len(chunks)}")

print("Summarizing chunks...")
final_summary = ""
for i, chunk in enumerate(chunks):
    print(f"Summarizing chunk {i+1}/{len(chunks)}...")
    summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
    final_summary += summary[0]['summary_text'] + " "

print("\nSummary ready.")

# STEP 5: Split summary into two parts for two voices
half = len(final_summary) // 2
text1 = final_summary[:half].strip()
text2 = final_summary[half:].strip()

print("Split summary into two parts for two voices.")

# STEP 6: ElevenLabs API details
api_key = "sk_528723c9ff2ed518d6e5b90e87a39e224f422860ae37e2a6"  # Your ElevenLabs API key here

# Your custom voice IDs (replace these with your actual voice IDs)
voice_id_1 = "cgSgspJ2msm6clMCkdW9"  # Voice 1 ID as string
voice_id_2 = "IKne3meq5aSn9XLyUdCD"  # Voice 2 ID as string

# Function to convert text to speech using ElevenLabs API
def text_to_speech(text, voice_id, filename):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {
        "xi-api-key": api_key,
        "Content-Type": "application/json"
    }
    data = {
        "text": text,
        "voice_settings": {
            "stability": 0.4,
            "similarity_boost": 0.75
        }
    }
    print(f"Generating speech for voice_id: {voice_id} ...")
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Saved audio to {filename}")
    else:
        print(f"Error generating speech: {response.status_code} - {response.text}")

# STEP 7: Generate audio files for both voices
text_to_speech(text1, voice_id_1, "voice1.mp3")
text_to_speech(text2, voice_id_2, "voice2.mp3")

# STEP 8: Merge audio files sequentially with a short pause
print("Merging audio files...")
audio1 = AudioSegment.from_file("voice1.mp3")
audio2 = AudioSegment.from_file("voice2.mp3")
pause = AudioSegment.silent(duration=700)  # 0.7 seconds pause

final_audio = audio1 + pause + audio2
final_audio.export("final_podcast.mp3", format="mp3")
print("Final podcast saved as 'final_podcast.mp3'.")

# STEP 9: Play the final podcast audio
ipd.display(ipd.Audio("final_podcast.mp3"))


Upload your PDF file:


Saving Uni.pdf to Uni (5).pdf
Extracting text from PDF...
Extracted 1880 characters of text.
Loading summarization model...


Device set to use cpu


Splitting text into chunks for summarization...
Total chunks to summarize: 2
Summarizing chunks...
Summarizing chunk 1/2...
Summarizing chunk 2/2...

Summary ready.
Split summary into two parts for two voices.
Generating speech for voice_id: cgSgspJ2msm6clMCkdW9 ...
Saved audio to voice1.mp3
Generating speech for voice_id: IKne3meq5aSn9XLyUdCD ...
Saved audio to voice2.mp3
Merging audio files...
Final podcast saved as 'final_podcast.mp3'.


In [ ]:
# Install required packages
!pip install pymupdf transformers requests pydub

from google.colab import files
import fitz
import requests
from transformers import pipeline
from pydub import AudioSegment
import IPython.display as ipd

# ========== STEP 1: Upload PDF ==========
print("Upload your Hindi PDF:")
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]

# ========== STEP 2: Extract text ==========
doc = fitz.open(pdf_filename)
text = ""
for page in doc:
    text += page.get_text()
print(f"\n✅ Extracted {len(text)} characters.")

# ========== STEP 3: Optional Summarization ==========
# Use a summarization model that supports Hindi or skip if not needed
# For now, we'll skip summarization and use full Hindi content

# Optional: If you want to chunk large text, split it here
half = len(text) // 2
text1 = text[:half].strip()
text2 = text[half:].strip()

# ========== STEP 4: ElevenLabs API Setup ==========
api_key = "sk_528723c9ff2ed518d6e5b90e87a39e224f422860ae37e2a6"  # Replace with your API key

# ⚠️ Replace these with actual Hindi-compatible voice IDs
voice_id_1 = "cgSgspJ2msm6clMCkdW9"  # e.g., "n5ya..."
voice_id_2 = "IKne3meq5aSn9XLyUdCD"  # e.g., "b6p3..."

# ========== STEP 5: Text-to-Speech Function ==========
def generate_speech(text, voice_id, filename):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {
        "xi-api-key": api_key,
        "Content-Type": "application/json"
    }
    payload = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.4,
            "similarity_boost": 0.8
        }
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"✅ Audio saved: {filename}")
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")

# ========== STEP 6: Generate Voice Files ==========
generate_speech(text1, voice_id_1, "hindi_voice1.mp3")
generate_speech(text2, voice_id_2, "hindi_voice2.mp3")

# ========== STEP 7: Merge Audio and Play ==========
audio1 = AudioSegment.from_file("hindi_voice1.mp3")
audio2 = AudioSegment.from_file("hindi_voice2.mp3")
pause = AudioSegment.silent(duration=700)

final_podcast = audio1 + pause + audio2
final_podcast.export("hindi_podcast.mp3", format="mp3")

print("✅ Final podcast created: hindi_podcast.mp3")
ipd.display(ipd.Audio("hindi_podcast.mp3"))


Upload your Hindi PDF:


Saving mb.pdf to mb.pdf

✅ Extracted 4759 characters.
✅ Audio saved: hindi_voice1.mp3
❌ Error: 401 - {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 10000. You have 33 credits remaining, while 2374 credits are required for this request."}}
✅ Final podcast created: hindi_podcast.mp3
